In [11]:
!python -V

Python 3.9.19


In [12]:
import pandas as pd

In [13]:
import pickle

In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [16]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/test/mlruns/1', creation_time=1716583599823, experiment_id='1', last_update_time=1716583599823, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [17]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [18]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [19]:
len(df_train), len(df_val)

(73908, 61921)

In [20]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [21]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [22]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

2024/05/25 04:28:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f58017cf0a104835bff3e88a013231ef', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2024/05/25 04:28:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/25 04:28:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715200888857

In [24]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [25]:
with mlflow.start_run():

    mlflow.set_tag("developer","mk")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha=0.1

    mlflow.log_param("alpha",alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse=mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse",rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin",artifact_path="models_pickle")



2024/05/25 04:28:57 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [26]:
import xgboost as xgb
from hyperopt import fmin,tpe,hp,STATUS_OK,Trials
from hyperopt.pyll import scope

In [27]:
train=xgb.DMatrix(X_train,label=y_train)
valid=xgb.DMatrix(X_val,label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:29:08] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.48707                          
[1]	validation-rmse:9.24475                           
[2]	validation-rmse:8.37161                           
[3]	validation-rmse:7.77070                           
[4]	validation-rmse:7.35947                           
[5]	validation-rmse:7.08160                           
[6]	validation-rmse:6.89256                           
[7]	validation-rmse:6.76188                           
[8]	validation-rmse:6.67548                           
[9]	validation-rmse:6.61423                           
  0%|          | 0/50 [00:15<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [30]:
params={
    "learning_rate": 0.4352204636204455,
    "max_depth": 26,
    "min_child_weight": 1.1725316520922004,
    "objective": "reg:linear",
    "reg_alpha": 0.2009313726687733,
    "reg_lambda": 0.14912784092631032, 
    "seed": 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

2024/05/25 04:29:30 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '240bffbf35884dde83b1421e7d3d693f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:29:36] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.98068
[1]	validation-rmse:7.57367
[2]	validation-rmse:7.00520
[3]	validation-rmse:6.78066
[4]	validation-rmse:6.67216
[5]	validation-rmse:6.61727
[6]	validation-rmse:6.58827
[7]	validation-rmse:6.57411
[8]	validation-rmse:6.55981
[9]	validation-rmse:6.55659
[10]	validation-rmse:6.55155
[11]	validation-rmse:6.54653
[12]	validation-rmse:6.54226
[13]	validation-rmse:6.53447
[14]	validation-rmse:6.53009
[15]	validation-rmse:6.52635
[16]	validation-rmse:6.52368
[17]	validation-rmse:6.52061
[18]	validation-rmse:6.51653
[19]	validation-rmse:6.51426
[20]	validation-rmse:6.51086
[21]	validation-rmse:6.50763
[22]	validation-rmse:6.50654
[23]	validation-rmse:6.50335
[24]	validation-rmse:6.50085
[25]	validation-rmse:6.49774
[26]	validation-rmse:6.49307
[27]	validation-rmse:6.49015
[28]	validation-rmse:6.48780
[29]	validation-rmse:6.48417
[30]	validation-rmse:6.47979
[31]	validation-rmse:6.47561
[32]	validation-rmse:6.47362
[33]	validation-rmse:6.47099
[34]	validation-rmse:6.4

2024/05/25 04:31:25 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/25 04:31:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:31:25] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


In [31]:
mlflow.xgboost.autolog(disable=True)

In [32]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params={
    "learning_rate": 0.4352204636204455,
    "max_depth": 26,
    "min_child_weight": 1.1725316520922004,
    "objective": "reg:linear",
    "reg_alpha": 0.2009313726687733,
    "reg_lambda": 0.14912784092631032, 
    "seed": 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


    

/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:32:06] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.98068
[1]	validation-rmse:7.57367
[2]	validation-rmse:7.00520
[3]	validation-rmse:6.78066
[4]	validation-rmse:6.67216
[5]	validation-rmse:6.61727
[6]	validation-rmse:6.58827
[7]	validation-rmse:6.57411
[8]	validation-rmse:6.55981
[9]	validation-rmse:6.55659
[10]	validation-rmse:6.55155
[11]	validation-rmse:6.54653
[12]	validation-rmse:6.54226
[13]	validation-rmse:6.53447
[14]	validation-rmse:6.53009
[15]	validation-rmse:6.52635
[16]	validation-rmse:6.52368
[17]	validation-rmse:6.52061
[18]	validation-rmse:6.51653
[19]	validation-rmse:6.51426
[20]	validation-rmse:6.51086
[21]	validation-rmse:6.50763
[22]	validation-rmse:6.50654
[23]	validation-rmse:6.50335
[24]	validation-rmse:6.50085
[25]	validation-rmse:6.49774
[26]	validation-rmse:6.49307
[27]	validation-rmse:6.49015
[28]	validation-rmse:6.48780
[29]	validation-rmse:6.48417
[30]	validation-rmse:6.47979
[31]	validation-rmse:6.47561
[32]	validation-rmse:6.47362
[33]	validation-rmse:6.47099
[34]	validation-rmse:6.4

/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:32:51] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [33]:
import mlflow
logged_model = 'runs:/958d6cd7394640739c766f2fb5296879/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:32:57] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [34]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 958d6cd7394640739c766f2fb5296879

In [35]:
xgboost_model=mlflow.xgboost.load_model(logged_model)

/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:33:10] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [36]:
y_pred=xgboost_model.predict(valid)

In [37]:
y_pred[:10]

array([14.384877 ,  6.971347 , 15.088541 , 24.5384   ,  9.416897 ,
       17.203144 , 10.735233 ,  7.5000467, 10.138649 , 16.955217 ],
      dtype=float32)

In [38]:
import mlflow
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [39]:
mlflow.sklearn.autolog(disable=True)

In [40]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/05/25 04:33:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/25 04:33:21 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/05/25 04:58:36 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/ubuntu/anaconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.